In [1]:
from PIL import Image, ImageEnhance, ImageFilter, ImageOps
import pytesseract, numpy as np
from scipy.ndimage import binary_dilation
from scipy.spatial import KDTree

# Make sure pytesseract knows where to find the Tesseract binary
pytesseract.pytesseract.tesseract_cmd = (
    r"C:\\Program Files\\Tesseract-OCR\\tesseract.exe"
)

We are going to use pyteseract to extract min and max temps so we have the scale range.

In [ ]:
# 1) Paths
INPUT_IMAGE  = "thermal_imges_for_exp\CH01.jpeg"
OUTPUT_IMAGE = "thermal_imges_for_exp\CH01_clean.jpeg"

# 2) OCR regions for extracting min/max temperatures
OCR_REGIONS = {
    "min_top_left": {"x": 36,  "y": 31,  "width": 34, "height": 15},
    "min_right":    {"x": 199, "y": 261, "width": 36, "height": 19},
    "max_top_left": {"x": 35,  "y": 45,  "width": 35, "height": 17},
    "max_right":    {"x": 197, "y": 38,  "width": 38, "height": 18},
}

# 3) Smoothing parameters
WHITE_TOLERANCE      = 150    # how “white” a pixel must be (0–255 scale)
NEIGHBOR_COUNT       = 150    # how many non‑white neighbors to average
WHITE_BOXES          = [     # only smooth white pixels inside these boxes
    {"x": -1,  "y": 0,   "width": 71, "height": 62},
    {"x": 166, "y": 3,   "width": 70, "height": 26},
    {"x": 197, "y": 36,  "width": 38, "height": 244},
    {"x": 177, "y": 299, "width": 54, "height": 21},
    {"x": 0,   "y": 302, "width": 46, "height": 17},
]
GREEN_BOX            = {"x": 99, "y": 140, "width": 38, "height": 41}
SMOOTH_TEMPERATURE_BAR = True
TEMPERATURE_BAR_BOX  = {"x": 215, "y": 56, "width": 20, "height": 209}

In [3]:
def run_ocr_on_coords(image_path, regions, conf_threshold=80):
    """
    Crops each named region, enhances contrast, sharpens, then runs OCR.
    Returns a dict with 'min' and 'max' values, their confidence scores,
    and boolean flags indicating if any conf < conf_threshold.
    """

    img = Image.open(image_path)
    raw_results = {}

    # 1) Run OCR on every region
    for label, box in regions.items():
        x, y, w, h = box["x"], box["y"], box["width"], box["height"]
        crop = img.crop((x, y, x + w, y + h))

        # Pre‑process: grayscale → boost contrast → sharpen
        gray     = ImageOps.grayscale(crop)
        enhanced = ImageEnhance.Contrast(gray).enhance(2.0)
        sharp    = enhanced.filter(ImageFilter.SHARPEN)

        # Get every detected fragment plus confidence
        data = pytesseract.image_to_data(
            sharp,
            output_type=pytesseract.Output.DICT,
            config="--psm 6 -c tessedit_char_whitelist=0123456789."
        )

        # Pick the fragment with highest confidence
        best_text, best_conf = "", -1
        for txt, conf in zip(data["text"], data["conf"]):
            try:
                c = float(conf)
            except ValueError:
                continue
            t = txt.strip()
            if t and c > best_conf:
                best_text, best_conf = t, c

        raw_results[label] = {"value": best_text, "conf": best_conf}

    # 2) If a value appears twice (top_left vs right), pick the higher‑confidence one
    def choose(a, b):
        r1, r2 = raw_results[a], raw_results[b]
        return r1 if r1["conf"] >= r2["conf"] else r2

    # Build final dict
    min_res = choose("min_top_left", "min_right")
    max_res = choose("max_top_left", "max_right")

    final = {
        "min":      min_res["value"],
        "min_conf": min_res["conf"],
        "min_flag": min_res["conf"] < conf_threshold,
        "max":      max_res["value"],
        "max_conf": max_res["conf"],
        "max_flag": max_res["conf"] < conf_threshold,
    }

    return final

In [4]:
def smooth_pixels(
    image_path, output_path,
    tolerance, neighbor_count,
    white_boxes, green_box,
    smooth_bar, bar_box
):
    """
    1) Finds all “near‑white” pixels (>= 255‑tolerance), dilates
       that mask by 2px in every direction.
    2) Only keeps those pixels inside white_boxes.
    3) For each, replaces its color by the average of its k nearest
       non‑white (background) pixels. Repeats twice for extra blur.
    4) Then does the same k‑NN blur for green pixels inside green_box.
    5) Optionally, blurs the temperature bar region inside bar_box.
    """
    # Load & convert to NumPy
    img     = Image.open(image_path).convert("RGB")
    arr     = np.array(img)
    h, w    = arr.shape[:2]
    white_t = 255 - tolerance

    # 1) Build and dilate the white mask
    white_mask = np.all(arr >= white_t, axis=-1)
    struct     = np.ones((5, 5), bool)  # 2px radius square
    dilated    = binary_dilation(white_mask, structure=struct)

    # 2) Keep only dilated pixels inside any white_box
    coords = np.argwhere(dilated)
    keep   = []
    for y, x in coords:
        for b in white_boxes:
            if (b["x"] <= x < b["x"]+b["width"]
            and b["y"] <= y < b["y"]+b["height"]):
                keep.append((y, x))
                break
    white_coords = np.array(keep)
    if white_coords.size == 0:
        print("No white pixels to smooth.")
        return 0

    # Prepare KD‑tree of all non‑white pixels
    non_white = np.argwhere(~dilated)
    tree      = KDTree(non_white)

    # 3) Two passes of k‑NN blur
    smoothed   = arr.copy()
    changed    = 0
    for _ in range(2):
        for y, x in white_coords:
            dists, idxs = tree.query((y, x), k=neighbor_count)
            nbrs        = non_white[idxs]
            colors      = arr[nbrs[:,0], nbrs[:,1]]
            avg_color   = colors.mean(axis=0).astype(np.uint8)
            if not np.array_equal(smoothed[y, x], avg_color):
                smoothed[y, x] = avg_color
                changed      += 1
        arr = smoothed.copy()  # use updated pixels for next pass

    # 4) Blur green pixels inside green_box
    gx, gy, gw, gh     = green_box["x"], green_box["y"], green_box["width"], green_box["height"]
    green_coords       = []
    green_box_mask     = np.zeros_like(white_mask)

    # Find green pixels in the specified box
    for i in range(gy, gy + gh):
        for j in range(gx, gx + gw):
            r, g, b = arr[i, j]
            if g > 100 and g > r and g > b:
                green_coords.append((i, j))
                green_box_mask[i, j] = True

    green_coords   = np.array(green_coords)
    outside_mask   = ~green_box_mask
    outside_coords = np.argwhere(outside_mask)

    if outside_coords.size > 0 and green_coords.size > 0:
        green_tree = KDTree(outside_coords)
        for y, x in green_coords:
            k = min(neighbor_count, len(outside_coords))
            dists, idxs     = green_tree.query((y, x), k=k)
            nearest_pixels  = outside_coords[idxs]
            pixel_values    = arr[nearest_pixels[:,0], nearest_pixels[:,1]]
            avg_color       = pixel_values.mean(axis=0).astype(np.uint8)
            smoothed[y, x]  = avg_color
            changed        += 1

    # 5) Blur the temperature bar if requested
    if smooth_bar:
        bx, by, bw, bh = bar_box.values()
        inside = [
            (y, x)
            for y in range(by, min(by+bh, h))
            for x in range(bx, min(bx+bw, w))
        ]
        outside_mask = ~(
            ( (np.arange(h)[:,None] >= by) & (np.arange(h)[:,None] < by+bh) )
            & ( (np.arange(w)[None,:] >= bx) & (np.arange(w)[None,:] < bx+bw) )
        )
        outside = np.argwhere(outside_mask)
        bar_tree = KDTree(outside)
        for y, x in inside:
            dists, idxs = bar_tree.query((y, x), k=min(neighbor_count, len(outside)))
            nbrs        = outside[idxs]
            avg_color   = smoothed[nbrs[:,0], nbrs[:,1]].mean(axis=0).astype(np.uint8)
            smoothed[y, x] = avg_color
            changed      += 1

        print("Temperature bar smoothed.")

    # Save and report
    Image.fromarray(smoothed).save(output_path)
    print(f"Saved smoothed image to {output_path}")
    return changed

In [5]:
if __name__ == "__main__":
    # 1) OCR step
    ocr_results = run_ocr_on_coords(INPUT_IMAGE, OCR_REGIONS)
    print("OCR results:", ocr_results)

    # 2) Smoothing step
    total_changed = smooth_pixels(
        INPUT_IMAGE, OUTPUT_IMAGE,
        WHITE_TOLERANCE, NEIGHBOR_COUNT,
        WHITE_BOXES, GREEN_BOX,
        SMOOTH_TEMPERATURE_BAR, TEMPERATURE_BAR_BOX
    )
    print(f"Total pixels changed: {total_changed}")

OCR results: {'min': '49.9', 'min_conf': 70.0, 'min_flag': True, 'max': '71.8', 'max_conf': 85.0, 'max_flag': False}
Temperature bar smoothed.
Saved smoothed image to thermal_imges_for_exp\CH01_666.jpeg
Total pixels changed: 10925
